In [1]:
# import re
import sys
import random
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from scipy import optimize
from gekko import GEKKO

def input_sisul():
    sisul = input('알아보고 싶은 시설명을 입력하세요:')
    temp = stage[stage['시설명'].str.contains(sisul)].copy()
    temp.reset_index(inplace=True, drop=True)
    
    if len(temp)==1:
        print(temp['시설명'])
        return temp.iloc[0, :]
    elif len(temp)==0:
        print('찾으시는 시설이 없습니다. 다른 시설을 입력하세요.')
        sys.exit()
    elif len(temp)>1:
        print(f'{len(temp)}개의 검색 결과가 있습니다.')
        print('원하는 공연장에 해당하는 숫자를 입력하세요:')
        for i in range(len(temp)):
            print(f"{i+1}. {temp['시설명'][i]} {temp['공연홀'][i]} ({temp['지역'][i]} {temp['시군구'][i]})")
        inp = input()
        try:
            hall = int(inp)
            return temp.iloc[hall-1, :]
        except:
            print('잘못된 입력입니다.')
            sys.exit()
            
def age_cat(x):
    try:
        x=int(x)
        if x<10: x='less than 10'
        elif x<20: x='10s'
        elif x<30: x='20s'
        elif x<40: x='30s'
        elif x<50: x='40s'
        elif x<60: x='50s'
        elif x<70: x='60s'
        elif x<80: x='70s'
        else: x='over 80'
        return x
    except:
        return x

def make_xy():
    temp = df.copy()
    temp = temp[temp['time']!='기타']
    # convert to binary classification
    temp['satisfaction'] = temp['satisfaction'].apply(lambda x: 1 if x>=5 else 0)
    
    temp = temp[['time', 'location', 'sex', 'age', 'genre', 'satisfaction']]
    #temp['age'] = temp['age'].apply(lambda x: age_cat(x))
    temp = pd.get_dummies(temp, columns=['location'])
    temp = pd.get_dummies(temp, columns=['time', 'age', 'sex', 'genre'], drop_first=True)

    X = temp.drop(columns=['satisfaction'])
    y = temp['satisfaction']
    return X, y

    
def get_coef(X, y):  
    # commit logsitic regression
    reg = LogisticRegression()
    reg.fit(X, y)
    cv_results = cross_validate(reg, X, y, cv=5, scoring='accuracy')
    print('logistic regression accuracy :')
    print(cv_results['test_score'].mean())
    return reg.coef_

# Optimization model of location and time with logistic regression model
def optimize(loc, sigungu, coef, X, k=-1):    
    m = GEKKO(remote=False)
    m.options.SOLVER=1  # APOPT is an MINLP solver

    mod = sys.modules[__name__]
    # create variables for location, time
    for i in range(20):
        setattr(mod, f'x{i}', m.Var(value=0,lb=0,ub=1,integer=True))
    # for age, sex
    for i in range(20, 28):
        setattr(mod, f'x{i}', m.Var(value=0,lb=0,ub=1))
    # for genre
    for i in range(28, 32):
        setattr(mod, f'x{i}', m.Var(value=0,lb=0,ub=1,integer=True))

    v1 = np.array([x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, #location
                   x17, x18, x19, #time
                   x20, x21, x22, x23, x24, x25, x26, #age
                   x27, #sex
                   x28, x29, x30, x31]) #genre
    loc_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울',
                '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
    try:
        temp = df_pop[(df_pop['지역']==loc) & (df_pop['시군구']==sigungu)]
    except:
        temp = df_pop[(df_pop['지역']==loc[0]) & (df_pop['시군구']==sigungu[0])]
    total = temp.iloc[:, 3:-1].sum().sum()
    sex_f = temp.iloc[1, 3:-1].sum() / total
    age_10 = temp.iloc[:, 4].sum() / total
    age_20 = temp.iloc[:, 5].sum() / total
    age_30 = temp.iloc[:, 6].sum() / total
    age_40 = temp.iloc[:, 7].sum() / total
    age_50 = temp.iloc[:, 8].sum() / total
    age_60 = temp.iloc[:, 9].sum() / total
    age_70 = temp.iloc[:, 10].sum() / total
    age_80 = temp.iloc[:, 11].sum() / total
    
    # set constraints
    m.Equation(x0+x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11+x12+x13+x14+x15+x16==1) # time: weekday daytime / weekday dinner / weekend daytime / weekend dinner
    m.Equation(x17+x18+x19<=1)
    m.Equation(eval('x'+str(loc_list.index(loc)))==1)
    m.Equation(x20==age_20)
    m.Equation(x21==age_30)
    m.Equation(x22==age_40)
    m.Equation(x23==age_50)
    m.Equation(x24==age_60)
    m.Equation(x25==age_70)
    m.Equation(x26==age_80)
    m.Equation(x27==sex_f)
    m.Equation(x28+x29+x30+x31<=1)
    # get next optimized value
    m.Equation(-(m.exp(np.dot(v1, coef[0]))) / (1+m.exp(np.dot(v1, coef[0])))>k)
    m.Obj(-(m.exp(np.dot(v1, coef[0]))) / (1+m.exp(np.dot(v1, coef[0]))))
    m.solve(disp=False)

    obj = (m.options.objfcnval)*-1
    lst = []
    time = '주말 낮 시간'
    for i in range(17, 19):
        if list(v1)[i][0] == 1:
            time = X.columns[i][5:]
    
    genre = 'acting'
    for i in range(28, 32):
        if list(v1)[i][0] == 1:
            genre = X.columns[i][6:]
    '''
    # find optimal age        
    max_age = 0
    for i in range(1, 9):
        age = eval('age_' + str(i) + '0')
        if max_age < age:
            max_age = age
            age_label = str(i) + '0대'
    # find optimal sex
    if sex_f >= 0.5:
        sex_label = '여성'
    else:
        sex_label = '남성'
    '''
    return time, genre, obj

# select the optimal stage under the conditions            
def show_select(sisul, genre: str, loc: str, time: str, pos):
    
    # find optimal age for given conditions
    tmp = df[(df['location']==loc) 
             & ~(df['satisfy_'+genre].isnull()) 
             & (df['time']==time)].groupby(['age', 'sex'])['location'].count()
    try:
        age, sex = tmp.idxmax()
    except:
        age, sex = '30대', '여자'
    # convert age label to Korean
    if age == '80s':
        age = '80대 이상'
    else:
        age = age.replace('s', '대')
    
    # convert genre label to Korean
    if genre == 'western':
        genre = '서양음악'
    elif genre == 'tradition':
        genre = '전통공연'
    elif genre == 'acting':
        genre = '연극'
    elif genre == 'musical':
        genre = '뮤지컬'
    elif genre == 'dancing':
        genre = '무용'
    
    output = pd.DataFrame([[sisul['시설명'], sisul['공연홀'], sisul['좌석수'], genre, time, age + ' ' + sex, pos]], 
                          columns=['시설명', '공연홀', '좌석수', '장르', '시간대', '타겟층', '만족도 점수'])
    return output


def genre_check(sisul, data):
    genres = ['서양음악', '전통공연', '연극', '뮤지컬', '무용']
    drop_index = []
    for genre in genres:
        if sisul[genre]==0:
            drop_index += list(data[data['장르']==genre].index)
    data = data.drop(index=drop_index)
    return data
            

df = pd.read_csv('data/data_regression.csv', index_col=0)
stage = pd.read_csv('data/stage_info.csv', index_col=0)
df_pop = pd.read_csv('data/data_population.csv', index_col=0)

sisul = input_sisul()
loc = sisul['지역']
sigungu = sisul['시군구']
dic = {}
X, y = make_xy()
coef = get_coef(X, y)
k = -1
for i in range(10):
    time, genre, pos = optimize(loc, sigungu, coef, X, k)
    if k==-1:
        output = show_select(sisul, genre, loc, time, pos)
    else:
        output = pd.concat([output, show_select(sisul, genre, loc, time, pos)])
    k = pos*-1 + 0.01

result = output.sort_values('만족도 점수', ascending=False).drop_duplicates()
result.reset_index(inplace=True, drop=True)
result = genre_check(sisul, result)
result.reset_index(inplace=True, drop=True)
result.index.name = '순위'
result.index += 1

result

알아보고 싶은 시설명을 입력하세요: 예술의전당


27개의 검색 결과가 있습니다.
원하는 공연장에 해당하는 숫자를 입력하세요:
1. 예술의전당 콘서트홀 (서울 서초구)
2. 예술의전당 오페라극장 (서울 서초구)
3. 예술의전당 IBK챔버홀 (서울 서초구)
4. 예술의전당 CJ 토월극장 (서울 서초구)
5. 예술의전당 리사이틀홀 (서울 서초구)
6. 예술의전당 자유소극장 (서울 서초구)
7. 예술의전당 신세계스퀘어 야외무대 (서울 서초구)
8. 대전예술의전당 아트홀 (대전 서구)
9. 대전예술의전당 앙상블홀 (대전 서구)
10. 안산문화예술의전당 해돋이극장 (경기 안산시)
11. 안산문화예술의전당 달맞이극장 (경기 안산시)
12. 안산문화예술의전당 별무리극장 (경기 안산시)
13. 경주예술의전당 대공연장(화랑홀) (경북 경주시)
14. 경주예술의전당 소공연장(원화홀) (경북 경주시)
15. 천안예술의전당 대공연장 (충남 천안시)
16. 천안예술의전당 소공연장 (충남 천안시)
17. 청주예술의전당 대공연장 (충북 청주시)
18. 청주예술의전당 소공연장 (충북 청주시)
19. 서귀포예술의전당 대극장 (제주 서귀포시)
20. 서귀포예술의전당 소극장 (제주 서귀포시)
21. 안동문화예술의전당 웅부홀 (경북 안동시)
22. 안동문화예술의전당 백조홀 (경북 안동시)
23. 계룡문화예술의전당 대공연장 (충남 계룡시)
24. 계룡문화예술의전당 소공연장(다목적홀) (충남 계룡시)
25. 영광예술의전당 대공연장 (전남 영광군)
26. 군산예술의전당 소공연장 (전북 군산시)
27. 군산예술의전당 대공연장 (전북 군산시)


 10


logistic regression accuracy :
0.8947283537897848


,시설명,공연홀,좌석수,장르,시간대,타겟층,만족도 점수
순위,,,,,,,
1,안산문화예술의전당,해돋이극장,1573,뮤지컬,주말 낮 시간,40대 여자,0.618403
2,안산문화예술의전당,해돋이극장,1573,뮤지컬,주말 저녁 시간,30대 여자,0.600187
3,안산문화예술의전당,해돋이극장,1573,뮤지컬,주말 낮 시간,40대 여자,0.570340
4,안산문화예술의전당,해돋이극장,1573,뮤지컬,주중 낮 시간,40대 여자,0.544208
5,안산문화예술의전당,해돋이극장,1573,무용,주말 낮 시간,20대 여자,0.481442
6,안산문화예술의전당,해돋이극장,1573,무용,주말 낮 시간,20대 여자,0.431974
7,안산문화예술의전당,해돋이극장,1573,무용,주중 낮 시간,40대 여자,0.406188
